# Intro
This is a notebook to work through David Hogg's "Data analysis recipes: Fitting a model to data" (https://arxiv.org/abs/1008.4686)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Section 1 - Standard Practice
### Exercise 1

In [ ]:
x = np.array([203, 58, 210, 202, 198, 158, 165, 201, 157,
              131, 166, 160, 186, 125, 218, 146]).astype(np.float64)
y = np.array([495, 173, 479, 504, 510, 416, 393, 442, 317,
              311, 400, 337, 423, 334, 533, 344]).astype(np.float64)
sigma_y = np.array([21, 15, 27, 14, 30, 16, 14, 25, 52,
                    16, 34, 31, 42, 26, 16, 22]).astype(np.float64)
covar = np.diag(sigma_y**2.0)
covar_inv = np.linalg.inv(covar)
A = np.stack([np.ones_like(x), x]).transpose()

In [ ]:
invmat = np.linalg.inv(np.dot(A.transpose(), np.dot(covar_inv, A)))

In [ ]:
bm = np.dot(invmat, np.dot(A.transpose(), np.dot(covar_inv, y)))

In [ ]:
# Print our answers for intercept (b) and slope (m)
bm

In [ ]:
# Plot the data and the fitted line
plt.errorbar(x, y, yerr=sigma_y, fmt='o')
plt.plot(np.arange(300), bm[1] * np.arange(300) + bm[0])
plt.xlim([0, 300]); plt.ylim([0, 700]);
plt.xlabel('x'); plt.ylabel('y')

In [ ]:
# invmat is the covariance matrix of our fit parameters.
# So the uncertainty variance, sigma^2_m, on the slope
# is the (1, 1) element of this matrix.
invmat[1, 1]

### Exercise 2
Add more data

In [ ]:
x = np.array([201, 244, 47, 287, 203, 58, 210, 202, 198, 158,
              165, 201, 157, 131, 166, 160, 186, 125, 218, 146]).astype(np.float64)
y = np.array([592, 401, 583, 402, 495, 173, 479, 504, 510,
              416, 393, 442, 317, 311, 400, 337, 423, 334, 533, 344]).astype(np.float64)
sigma_y = np.array([61, 25, 38, 15, 21, 15, 27, 14, 30, 16,
                    14, 25, 52, 16, 34, 31, 42, 26, 16, 22]).astype(np.float64)
covar = np.diag(sigma_y**2.0)
covar_inv = np.linalg.inv(covar)
A = np.stack([np.ones_like(x), x]).transpose()

In [ ]:
invmat = np.linalg.inv(np.dot(A.transpose(), np.dot(covar_inv, A)))
bm = np.dot(invmat, np.dot(A.transpose(), np.dot(covar_inv, y)))
print(bm)

In [ ]:
# Plot the data and the fitted line
plt.errorbar(x, y, yerr=sigma_y, fmt='o')
plt.plot(np.arange(300), bm[1] * np.arange(300) + bm[0])
plt.xlim([0, 300]); plt.ylim([0, 700]);
plt.xlabel('x'); plt.ylabel('y')

In [ ]:
# Uncertainty variance on slope:
invmat[1, 1]

So what we see here is that the new data points (four of them) seem to
not agree with the other 16 points, causing the data to look more scattered.
However, they have very small error bars, and so the uncertainty on the
fit dropped.

### Exercise 3
Generalize to quadratic

In [ ]:
x = np.array([203, 58, 210, 202, 198, 158, 165, 201, 157,
              131, 166, 160, 186, 125, 218, 146]).astype(np.float64)
y = np.array([495, 173, 479, 504, 510, 416, 393, 442, 317,
              311, 400, 337, 423, 334, 533, 344]).astype(np.float64)
sigma_y = np.array([21, 15, 27, 14, 30, 16, 14, 25, 52,
                    16, 34, 31, 42, 26, 16, 22]).astype(np.float64)
covar = np.diag(sigma_y**2.0)
covar_inv = np.linalg.inv(covar)
A = np.stack([np.ones_like(x), x, x**2.0]).transpose()

In [ ]:
invmat = np.linalg.inv(np.dot(A.transpose(), np.dot(covar_inv, A)))
bmq = np.dot(invmat, np.dot(A.transpose(), np.dot(covar_inv, y)))
print(bmq)

In [ ]:
# Plot the data and the fitted line
plt.errorbar(x, y, yerr=sigma_y, fmt='o')
plt.plot(np.arange(300), bmq[2] * np.arange(300)**2.0 + bmq[1] * np.arange(300) + bmq[0])
plt.xlim([0, 300]); plt.ylim([0, 700]);
plt.xlabel('x'); plt.ylabel('y')

## Section 1 - The Objective Function
### Exercise 4

Each of our data points can be described with the frequency distribution
$$p(t_i|\sigma_{ti}, T) = \frac{1}{\sqrt{2\pi \sigma_{ti}^2}} \mathrm{exp}\left(-\frac{[t_i - T]^2}{2\sigma_{ti}^2}\right) $$
And the likelihood is the product of these probabilities.
$$\mathcal{L} = \prod_{i=1}^{N} p(t_i|\sigma_{ti}, T)$$
The log likelihood is then
$$\ln\mathcal{L} = K - \sum_{i=1}^N \frac{[t_i - T]^2}{2\sigma_{ti}^2}$$

We then take a derivative of $\ln \mathcal{L}$ with respect to $T$ and set equal to zero to maximize the likelihood.
$$ \frac{\partial \ln \mathcal{L}}{\partial T} = \sum_{i=1}^N \frac{t_i-T}{\sigma_{ti}^2} = 0$$
$$ \sum_{i=1}^N \frac{t_i}{\sigma_{ti}^2} = \sum_{i=1}^N \frac{T}{\sigma_{ti}^2} $$
$$ \sum_{i=1}^N \frac{t_i}{\sigma_{ti}^2} = T \sum_{i=1}^N \frac{1}{\sigma_{ti}^2} $$
Dividing by the sum on the right hand side, we arrive at our result, which is simply a weighted mean, where the weights are given by the inverse of the variances.
$$ \frac{\sum_{i=1}^N \frac{t_i}{\sigma_{ti}^2}}{\sum_{i=1}^N \frac{1}{\sigma_{ti}^2} } = T $$

### Exercise 5
Start with the matrix expression for $\chi^2$.
$$ \chi^2 = [\mathbf{Y} - \mathbf{A X}]^T \mathbf{C}^{-1} [\mathbf{Y} - \mathbf{A X}]$$
In order to more clearly see how the matrix multiplications, transforms, etc work, I will use Einstein notation. Repeated indices are summed, and unpaired indices are the indices of the total expression. I will also drop the boldface just to save on my own typing.
$$ \chi^2 = [Y_i - A_{ij} X_j] C^{-1}_{ik} [Y_k - A_{kl} X_l] $$

Now we take a derivative with respect to one component of $X$. I will use yet another index, say $a$. Now, because $a$ will be unpaired, the result will actually be a vector, and we can read off each component of $X$ through $a$. Note that even though $X_a$ does not explicitly appear in our expression for $\chi^2$, $X_j$ and $X_l$ will both assume the value of $X_a$ within their sums. As a result, taking a derivative will pick up the terms when $j=a$ and when $l=a$. After employing the produce rule of derivatives, we end up with two terms (which are really more than two terms because they each contain sums).
$$\frac{\partial \chi^2}{\partial X_a} = -A_{ia}C^{-1}_{ik}[Y_k-A_{kl}X_l] - [Y_i-A_{ij}X_j]C^{-1}_{ik}A_{ka} $$

Now we will see the beauty of using Einstein notation to handle our matrix operations. Even though matrix multiplications do not in general commute, in this form we can readily move terms around as long as we do not break any summation rules. So I will rearrange the second term to look like the first.
$$ -A_{ia}C^{-1}_{ik}[Y_k-A_{kl}X_l] - [Y_i-A_{ij}X_j]C^{-1}_{ik}A_{ka} =  -A_{ia}C^{-1}_{ik}[Y_k-A_{kl}X_l] - A_{ka}C^{-1}_{ik}[Y_i-A_{ij}X_j]$$

Again, using the wonderfulness of Einstein notation, we switch around some of the "dummy indices" - indices which are summed over, so they do not carry any meaning outside their own term. In the first term, I will replace $k$ with $i$, $l$ with $j$, and $i$ with $k$.
$$ -A_{ia}C^{-1}_{ik}[Y_k-A_{kl}X_l] - A_{ka}C^{-1}_{ik}[Y_i-A_{ij}X_j] = -A_{ka}C^{-1}_{ki}[Y_i-A_{ij}X_j] - A_{ka}C^{-1}_{ik}[Y_i-A_{ij}X_j]$$

Finally, we note that $C^{-1}$ is symmetric, so we can exchange its indices freely. This gets us half way to our result.
$$ -A_{ka}C^{-1}_{ki}[Y_i-A_{ij}X_j] - A_{ka}C^{-1}_{ik}[Y_i-A_{ij}X_j] = -A_{ka}C^{-1}_{ki}[Y_i-A_{ij}X_j] - A_{ka}C^{-1}_{ki}[Y_i-A_{ij}X_j] $$
$$ = -2 A_{ka}C^{-1}_{ki}[Y_i-A_{ij}X_j] $$
And setting our derivative equal to zero,
$$ \frac{\partial \chi^2}{\partial X_a} = -2 A_{ka}C^{-1}_{ki}[Y_i-A_{ij}X_j] = 0 $$

At this stage (and it takes practice to know when to do this), it is convenient to switch back to matrix notation. But I'm still not going to use boldface.
$$ A^T C^{-1} [Y - AX] = 0$$

The remaining steps are to distribute the sum, move one term to the RHS, and insert some matrix inverses to solve for $X$.
$$ A^TC^{-1}Y - A^T C^{-1} A X = 0 $$
$$ A^T C^{-1} Y = A^T C^{-1} A X $$
$$ [A^T C^{-1} A]^{-1} [A^T C^{-1} Y] = [A^T C^{-1} A]^{-1} A^T C^{-1} A X $$
$$ [A^T C^{-1} A]^{-1} [A^T C^{-1} Y] = X $$

This is the result we started wtih, Eq. 5.